In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from solve_network import palette
import yaml
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter

In [ ]:
def load_configuration(config_path):
    """
    Load configuration settings from a YAML file.
    """
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config


config = load_configuration("../config_dash3.yaml")

run = "manuscript-dashboard3"  # run name from config.yaml

In [ ]:
datacenters = config["ci"]["datacenters"]  # pair name from config.yaml
locations = list(datacenters.keys())
names = list(datacenters.values())
zone = config["scenario"]["zone"][0]  # zone name from config.yaml

scaling = int(config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H

participation = config["scenario"]["participation"]
palette_techs = palette("p4")
year = config["scenario"]["year"][0]


(
    clean_techs,
    storage_techs,
    storage_charge_techs,
    storage_discharge_techs,
) = palette_techs

# renaming technologies for plotting
clean_chargers = [tech.replace(" ", "_") for tech in storage_charge_techs]
clean_dischargers = [tech.replace(" ", "_") for tech in storage_discharge_techs]


def tech_names(base_names, year):
    return [f"{name.replace(' ', '_')}-{year}" for name in base_names]


# expected technology names with year
exp_generators = tech_names(["offwind-ac", "offwind-dc", "onwind", "solar"], year)
exp_links = tech_names(["OCGT"], year)
exp_chargers = tech_names(
    ["battery charger", "H2 Electrolysis", "ironair discharger"], year
)
exp_dischargers = tech_names(
    ["battery discharger", "H2 Fuel Cell", "ironair charger"], year
)

# Assign colors
tech_colors = load_configuration("../config.yaml")["tech_colors"]

# Rename mappings
rename_ci_cost = pd.Series(
    {
        "onwind": "onshore wind",
        "solar": "solar",
        "grid": "grid imports",
        "revenue": "revenue",
        "battery_storage": "battery",
        "battery_inverter": "battery",
        "battery_charger": "battery",
        "ironair_storage": "ironair",
        "ironair_inverter": "ironair",
        "ironair_charger": "ironair",
        "hydrogen_storage": "hydrogen storage",
        "hydrogen_electrolysis": "hydrogen storage",
        "hydrogen_fuel_cell": "hydrogen storage",
        "adv_geothermal": "advanced dispatchable",
        "allam_ccs": "NG-Allam",
    }
)

rename_ci_capacity = pd.Series(
    {
        "onwind": "onshore wind",
        "solar": "solar",
        "battery_discharger": "battery",
        "battery_charger": "battery",
        "ironair_discharger": "ironair",
        "ironair_charger": "ironair",
        "H2_Fuel_Cell": "hydrogen fuel cell",
        "H2_Electrolysis": "hydrogen electrolysis",
        "adv_geothermal": "advanced dispatchable",
        "allam_ccs": "NG-Allam",
    }
)

system_techs = {
    "offwind-ac": "wind",
    "offwind-dc": "wind",
    "Offshore Wind": "wind",
    "onwind": "wind",
    "Onshore Wind": "wind",
    "solar": "solar PV",
    "Solar": "solar PV",
    "OCGT": "gas OC",
    "CCGT": "gas CC",
    "coal": "hard coal",
    "lignite": "lignite",
    "oil": "oil",
    "urban central solid biomass CHP": "biomass",
    "Pumped Hydro Storage": "PHS",
    "Reservoir & Dam": "hydro",
    "battery": "battery storage",
    "battery_discharger": "battery storage",
    "H2_Fuel_Cell": "hydrogen fuel cell",
    "H2_Electrolysis": "hydrogen electrolysis",
    "ironair_discharger": "ironair storage",
    "ironair": "ironair storage",
}

rename_system_simple = {
    f"{key}-%s" % year: value for key, value in system_techs.items()
}

preferred_order = pd.Index(
    [
        "advanced dispatchable",
        "NG-Allam",
        "biomass",
        "hard coal",
        "lignite",
        "gas OC",
        "gas CC",
        "oil",
        "offshore wind",
        "onshore wind",
        "wind",
        "solar PV",
        "PHS",
        "battery",
        "battery storage",
        "ironair storage",
        "hydrogen storage",
        "hydrogen electrolysis",
        "hydrogen fuel cell",
    ]
)

In [ ]:
def rename_scen(obj, level=0):
    """
    Automatically append a percent sign to numeric string column names if not already present and rename DataFrame columns.

    Returns:
    - None: Modifies the DataFrame in place.
    """
    if isinstance(obj, pd.DataFrame):
        # Handle DataFrame: Modify column names
        modified_dict = {
            key: f"{key}%" if key.isdigit() and "%" not in key else key
            for key in obj.columns.get_level_values(level)
        }
        obj.rename(columns=modified_dict, level=level, inplace=True)

    elif isinstance(obj, pd.Series):
        # Handle Series: Modify index labels
        modified_dict = {
            key: f"{key}%" if "%" not in key else key
            for key in obj.index.get_level_values(level)
        }
        obj.rename(index=modified_dict, level=level, inplace=True)

### collect data

In [ ]:
import glob


def find_summary_files(run):
    path_pattern = f"../results/{run}/**/summary.csv"
    summary_files = glob.glob(path_pattern, recursive=True)
    return summary_files


summary_files = find_summary_files(run)

In [ ]:
summary_files

In [ ]:
# concatenate all summary files into a single dataframe
df = pd.concat(
    [pd.read_csv(f, index_col=0, header=[0, 1]) for f in summary_files],
    axis=1,
    keys=["-".join(f.split("/")[-3:-1]) for f in summary_files],
)

In [ ]:
# drop country level
df.columns = df.columns.droplevel([0, 2])
df

### Plotting

In [ ]:
FONTSIZE = 14

C0 = (
    config["costs"][f"ironair_energy_{year}"]
    * config["costs"]["USD2023_to_EUR2023"]
    * config["costs"]["ironair_duration"]
    + config["costs"][f"ironair_capacity_{year}"]
    * config["costs"]["USD2023_to_EUR2023"]
)

In [ ]:
def system_investment(
    df, tech_colors, rename_system_simple, preferred_order, year, C0=C0
):
    # Collect data
    gens = df.loc[["system_inv_" + t.replace(" ", "_") for t in exp_generators]].rename(
        {"system_inv_" + t.replace(" ", "_"): t for t in exp_generators}
    )

    links = df.loc[["system_inv_" + t.replace(" ", "_") for t in exp_links]].rename(
        {"system_inv_" + t.replace(" ", "_"): t for t in exp_links}
    )

    dischargers = df.loc[
        ["system_inv_" + t.replace(" ", "_") for t in exp_dischargers]
    ].rename({"system_inv_" + t.replace(" ", "_"): t for t in exp_dischargers})

    chargers = df.loc[
        ["system_inv_" + t.replace(" ", "_") for t in exp_chargers]
    ].rename({"system_inv_" + t.replace(" ", "_"): t for t in exp_chargers})

    chargers = chargers.drop(["battery_charger-%s" % year])

    # Combine all components into a unified DataFrame
    ldf = pd.concat([gens, links, dischargers, chargers], axis=0)

    # Drop rows where all values are below a certain threshold
    to_drop = ldf.index[(ldf < 0.1).all(axis=1)]
    ldf.drop(to_drop, inplace=True)

    # Rename scenario names and group by the simpler names
    rename_scen(ldf, level=0)
    ldf = ldf.groupby(rename_system_simple).sum()

    # Reorder based on preferred order
    new_index = preferred_order.intersection(ldf.index).append(
        ldf.index.difference(preferred_order)
    )
    ldf = ldf.loc[new_index]

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    (ldf / 1e3).T.plot(
        kind="bar",
        stacked=True,
        ax=ax,
        color=tech_colors,
        width=0.65,
        edgecolor="black",
        linewidth=0.05,
    )

    # def _format_label(label):
    #     percentage_value = float(label.strip("%")) / 100  # Convert to decimal
    #     capex_value = C0 * percentage_value
    #     return int(capex_value)  # Format as integer EUR/kW

    # Formatting
    ax.grid(alpha=0.8, which="both", linestyle="--")
    ax.set_axisbelow(True)
    ax.tick_params(axis="x", labelsize=FONTSIZE)
    ax.tick_params(axis="y", labelsize=FONTSIZE)
    ax.set_ylabel(f"power capacity [GW]", fontsize=FONTSIZE)
    ax.set_title(f"Energy technology investments by {year}", fontsize=16, weight="bold")
    ax.legend(loc="lower right", ncol=2, prop={"size": FONTSIZE})

    # Handle emissions data for secondary axis
    emissions_data = df.loc["emissions_zone"]
    rename_scen(emissions_data)

    # Create secondary y-axis
    ax2 = ax.twinx()
    emissions_data.plot(
        kind="line",
        ax=ax2,
        color="black",
        linestyle="--",
        marker="o",
        linewidth=1.5,
        markersize=5,
    )

    # Formatting secondary axis
    ax2.set_ylabel(r"Emissions in local zone [MtCO$_2\cdot a^{-1}$]", fontsize=FONTSIZE)
    # set min of y axis to 0
    ax2.set_ylim(bottom=0)
    ax2.tick_params(axis="y", labelsize=12)
    ax2.grid(False)  # Turn off grid for the secondary axis

    ax.set_xticklabels(
        [
            f"{col} CAPEX \n {int(float(col.strip('%'))/100*C0)} EUR/kW"
            for col in ldf.columns.tolist()
        ],
        rotation=0,
        ha="center",
        fontsize=12,
    )

    fig.tight_layout()
    fig.savefig(
        "../manuscript/images/dashboard_3.pdf", transparent=True, bbox_inches="tight"
    )


system_investment(
    df=df,
    tech_colors=tech_colors,
    rename_system_simple=rename_system_simple,
    preferred_order=preferred_order,
    year=year,
)

In [ ]:
df.loc["system_emissions"] / 1e6

In [ ]:
350 * 1e6 / (1686 * 1e3)